In [1]:
import requests


In [2]:
import sys
sys.path.append('../')

In [3]:
import os
os.chdir("..")

In [4]:

from api import api
import pandas as pd

In [5]:
def reset(uid):
    url = f'http://127.0.0.1:5000/reset/experiment/{uid}'
    response = requests.get(url)
    response
    

def query(uid):
    url = f'http://127.0.0.1:5000/query/experiment/{uid}'
    video_pair = requests.get(url)
    vid_pair =video_pair.json()
    try:
        vid_pair = [v.split("/")[-2] for v in vid_pair]
    except:
        return vid_pair
    return vid_pair

def submit(vid_pair, pref_vid, uid):
    url = f'http://127.0.0.1:5000/submit/experiment/{uid}'
    # pref_vid = vid_pair[1]
    response = requests.post(url, data={
        "video1": vid_pair[0], 
        "video2": vid_pair[1],
        "pref_vid": pref_vid
    })
    return response

In [6]:
import random
from tqdm.notebook import tqdm
import time 

In [7]:
uid_list = [f"U{i:02d}" for i in range(2, 43, 2)]
for uid in uid_list:
    reset(uid)
    # # sleep for 1 second
    # time.sleep(0.01)

In [8]:
def simulater_uid_list(uid_list):
    for uid in uid_list:
        reset(uid)

    n_iter = 120*len(uid_list)
    with tqdm(range(n_iter), total=n_iter) as pbar:
        for i in pbar:
            uid = random.choice(uid_list)
            vid_pair = query(uid)
            if isinstance(vid_pair, str):
                print(vid_pair)
                break
            pref_vid = random.choice(vid_pair + ["no pref"])
            pbar.set_postfix({"pref_vid": pref_vid})
            response = submit(vid_pair=vid_pair, pref_vid=pref_vid, uid=uid)
            if response.status_code != 200:
                print(i, response.text)
                break
            if response.text != f"database/experiment/{uid}_pref.csv":
                print(i, response.text)
                if response.text == '"No more video pairs to label."\n':
                    uid_list.remove(uid)
                    if len(uid_list) == 0:
                        break

In [9]:
uid_list = [f"U{i:02d}" for i in range(2, 43, 2)]
uid_lists = [uid_list[i:i+3] for i in range(0, len(uid_list), 3)]

In [10]:
# for uid_list in uid_lists:
#     simulater_uid_list(uid_list)
simulater_uid_list(uid_list)

  0%|          | 0/2520 [00:00<?, ?it/s]

163 "Feeling tried? Take a break if necessary and please stay attentive in the following sessions."

179 "According to our record so far, you have been rather careful and thorough in the past labeling sessions! \nGood job! Take a break if needed and keep on the good work."

200 "Feeling tried? Take a break if necessary and please stay attentive in the following sessions."

235 "According to our record so far, you have been rather careful and thorough in the past labeling sessions! \nGood job! Take a break if needed and keep on the good work."

241 "According to our record so far, you have been rather careful and thorough in the past labeling sessions! \nGood job! Take a break if needed and keep on the good work."

253 "Feeling tried? Take a break if necessary and please stay attentive in the following sessions."

275 "Feeling tried? Take a break if necessary and please stay attentive in the following sessions."

292 "According to our record so far, you have been rather careful and thor

In [11]:
# reset("U02")
# reset("U04")

In [12]:
# uid_list = ["U02", "U04"]
# with tqdm(range(240), total=240) as pbar:
#     for i in pbar:
#         uid = random.choice(uid_list)
#         vid_pair = query(uid)
#         if isinstance(vid_pair, str):
#             print(vid_pair)
#             break
#         pref_vid = random.choice(vid_pair + ["no pref"])
#         pbar.set_postfix({"pref_vid": pref_vid})
#         response = submit(vid_pair=vid_pair, pref_vid=pref_vid, uid=uid)
#         if response.status_code != 200:
#             print(i, response.text)
#             break
#         if response.text != f"database/experiment/{uid}_pref.csv":
#             print(i, response.text)
#             if response.text == '"No more video pairs to label."\n':
#                 uid_list.remove(uid)
#                 if len(uid_list) == 0:
#                     break

In [13]:
# with tqdm(range(2500), total=2500) as pbar:
#     for i in pbar:
#         uid = random.choice(uid_list)
#         vid_pair = query(uid)
#         if isinstance(vid_pair, str):
#             print(vid_pair)
#             break
#         pref_vid = random.choice(vid_pair + ["no pref"])
#         pbar.set_postfix({"pref_vid": pref_vid})
#         response = submit(vid_pair=vid_pair, pref_vid=pref_vid, uid=uid)
#         if response.status_code != 200:
#             print(i, response.text)
#             break
#         if response.text != f"database/experiment/{uid}_pref.csv":
#             print(i, response.text)
#             if response.text == '"No more video pairs to label."\n':
#                 uid_list.remove(uid)
#                 if len(uid_list) == 0:
#                     break

In [14]:
import pandas as pd

In [15]:
uid_list = [f"U{i:02d}" for i in range(2, 43, 2)]

In [16]:
dfs = []
for uid in uid_list:
    df_uid = pd.read_csv(f"database/history/{uid}_history.csv")
    df_uid["user_id"] = uid
    dfs.append(df_uid)

In [17]:
df_labeled = []
for uid in uid_list:
    df_labeled_uid = pd.read_csv(f"database/experiment/{uid}_pref.csv")
    df_labeled_uid = df_labeled_uid[df_labeled_uid["pref"] != -1].copy()
    df_labeled_uid["user_id"] = uid
    df_labeled.append(df_labeled_uid)

In [18]:
df_history = pd.concat(dfs)
df_labeled_all = pd.concat(df_labeled)

In [19]:
df_history['pair_id'].value_counts().value_counts()

count
5    405
6     30
Name: count, dtype: int64

In [20]:
all_user_pref = pd.read_csv("database/experiment/all_user_pref.csv")

In [21]:
len(df_history), len(df_labeled_all), len(all_user_pref)

(2205, 2205, 2415)